Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 2: ES

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The goal of the game is to **avoid** taking the last object.

* Task2.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task2.2: An agent using evolved rules using ES

## Instructions

* Create the directory `lab2` inside the course repo 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.



In [2]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
import random
from copy import deepcopy


## The *Nim* and *Nimply* classes

In [3]:
Nimply = namedtuple("Nimply", "row, num_objects")


In [4]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects


## Sample (and silly) startegies 

In [5]:
def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)


In [6]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))


In [7]:
def adaptive(state: Nim) -> Nimply:
    """A strategy that can adapt its parameters"""
    genome = {"love_small": 0.5}


In [8]:
import numpy as np


def calculate_nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)


def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = calculate_nim_sum(tmp)
    return cooked


def optimal(state: Nim) -> Nimply:
    analysis = analize(state)
    print(analysis["possible_moves"].items())
    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    ply = random.choice(spicy_moves)
    return ply


## Oversimplified match

In [9]:
logging.getLogger().setLevel(logging.INFO)

strategy = (gabriele,optimal)


player = 0
won={"0": 0,"1":0}
for _ in range(20):
    nim = Nim(6)
    logging.info(f"init : {nim}")
    print(nim.rows)
    while nim:
        ply = strategy[player](nim)
        logging.info(f"ply: player {player} plays {ply}")
        nim.nimming(ply)
        logging.info(f"status: {nim}")
        player = 1 - player
    won[str(player)]+=1
    logging.info(f"status: Player {player} won!")
print("Winning games : ",won)

INFO:root:init : <1 3 5 7 9 11>
INFO:root:ply: player 0 plays Nimply(row=0, num_objects=1)
INFO:root:status: <0 3 5 7 9 11>
INFO:root:ply: player 1 plays Nimply(row=3, num_objects=6)
INFO:root:status: <0 3 5 1 9 11>
INFO:root:ply: player 0 plays Nimply(row=1, num_objects=3)
INFO:root:status: <0 0 5 1 9 11>
INFO:root:ply: player 1 plays Nimply(row=5, num_objects=11)
INFO:root:status: <0 0 5 1 9 0>
INFO:root:ply: player 0 plays Nimply(row=2, num_objects=5)
INFO:root:status: <0 0 0 1 9 0>
INFO:root:ply: player 1 plays Nimply(row=4, num_objects=6)
INFO:root:status: <0 0 0 1 3 0>
INFO:root:ply: player 0 plays Nimply(row=3, num_objects=1)
INFO:root:status: <0 0 0 0 3 0>
INFO:root:ply: player 1 plays Nimply(row=4, num_objects=1)
INFO:root:status: <0 0 0 0 2 0>
INFO:root:ply: player 0 plays Nimply(row=4, num_objects=2)
INFO:root:status: <0 0 0 0 0 0>
INFO:root:status: Player 1 won!
INFO:root:init : <1 3 5 7 9 11>
INFO:root:ply: player 1 plays Nimply(row=0, num_objects=1)
INFO:root:status: <0 3

(1, 3, 5, 7, 9, 11)
dict_items([(Nimply(row=1, num_objects=1), 2), (Nimply(row=1, num_objects=2), 1), (Nimply(row=1, num_objects=3), 0), (Nimply(row=2, num_objects=1), 2), (Nimply(row=2, num_objects=2), 5), (Nimply(row=2, num_objects=3), 4), (Nimply(row=2, num_objects=4), 7), (Nimply(row=2, num_objects=5), 6), (Nimply(row=3, num_objects=1), 2), (Nimply(row=3, num_objects=2), 1), (Nimply(row=3, num_objects=3), 0), (Nimply(row=3, num_objects=4), 7), (Nimply(row=3, num_objects=5), 6), (Nimply(row=3, num_objects=6), 5), (Nimply(row=3, num_objects=7), 4), (Nimply(row=4, num_objects=1), 2), (Nimply(row=4, num_objects=2), 13), (Nimply(row=4, num_objects=3), 12), (Nimply(row=4, num_objects=4), 15), (Nimply(row=4, num_objects=5), 14), (Nimply(row=4, num_objects=6), 9), (Nimply(row=4, num_objects=7), 8), (Nimply(row=4, num_objects=8), 11), (Nimply(row=4, num_objects=9), 10), (Nimply(row=5, num_objects=1), 2), (Nimply(row=5, num_objects=2), 1), (Nimply(row=5, num_objects=3), 0), (Nimply(row=5, nu

INFO:root:ply: player 0 plays Nimply(row=3, num_objects=7)
INFO:root:status: <0 0 0 0 3 1>
INFO:root:ply: player 1 plays Nimply(row=5, num_objects=1)
INFO:root:status: <0 0 0 0 3 0>
INFO:root:ply: player 0 plays Nimply(row=4, num_objects=3)
INFO:root:status: <0 0 0 0 0 0>
INFO:root:status: Player 1 won!
INFO:root:init : <1 3 5 7 9 11>
INFO:root:ply: player 1 plays Nimply(row=4, num_objects=3)
INFO:root:status: <1 3 5 7 6 11>
INFO:root:ply: player 0 plays Nimply(row=0, num_objects=1)
INFO:root:status: <0 3 5 7 6 11>
INFO:root:ply: player 1 plays Nimply(row=2, num_objects=1)
INFO:root:status: <0 3 4 7 6 11>
INFO:root:ply: player 0 plays Nimply(row=1, num_objects=3)
INFO:root:status: <0 0 4 7 6 11>
INFO:root:ply: player 1 plays Nimply(row=5, num_objects=11)
INFO:root:status: <0 0 4 7 6 0>
INFO:root:ply: player 0 plays Nimply(row=2, num_objects=4)
INFO:root:status: <0 0 0 7 6 0>
INFO:root:ply: player 1 plays Nimply(row=3, num_objects=6)
INFO:root:status: <0 0 0 1 6 0>
INFO:root:ply: player

dict_items([(Nimply(row=4, num_objects=1), 3), (Nimply(row=4, num_objects=2), 0), (Nimply(row=4, num_objects=3), 1), (Nimply(row=5, num_objects=1), 3)])
(1, 3, 5, 7, 9, 11)
dict_items([(Nimply(row=0, num_objects=1), 3), (Nimply(row=1, num_objects=1), 3), (Nimply(row=1, num_objects=2), 0), (Nimply(row=1, num_objects=3), 1), (Nimply(row=2, num_objects=1), 3), (Nimply(row=2, num_objects=2), 4), (Nimply(row=2, num_objects=3), 5), (Nimply(row=2, num_objects=4), 6), (Nimply(row=2, num_objects=5), 7), (Nimply(row=3, num_objects=1), 3), (Nimply(row=3, num_objects=2), 0), (Nimply(row=3, num_objects=3), 1), (Nimply(row=3, num_objects=4), 6), (Nimply(row=3, num_objects=5), 7), (Nimply(row=3, num_objects=6), 4), (Nimply(row=3, num_objects=7), 5), (Nimply(row=4, num_objects=1), 3), (Nimply(row=4, num_objects=2), 12), (Nimply(row=4, num_objects=3), 13), (Nimply(row=4, num_objects=4), 14), (Nimply(row=4, num_objects=5), 15), (Nimply(row=4, num_objects=6), 8), (Nimply(row=4, num_objects=7), 9), (Nimpl

INFO:root:status: <0 0 0 0 1 7>
INFO:root:ply: player 0 plays Nimply(row=4, num_objects=1)
INFO:root:status: <0 0 0 0 0 7>
INFO:root:ply: player 1 plays Nimply(row=5, num_objects=3)
INFO:root:status: <0 0 0 0 0 4>
INFO:root:ply: player 0 plays Nimply(row=5, num_objects=4)
INFO:root:status: <0 0 0 0 0 0>
INFO:root:status: Player 1 won!
INFO:root:init : <1 3 5 7 9 11>
INFO:root:ply: player 1 plays Nimply(row=3, num_objects=3)
INFO:root:status: <1 3 5 4 9 11>
INFO:root:ply: player 0 plays Nimply(row=0, num_objects=1)
INFO:root:status: <0 3 5 4 9 11>
INFO:root:ply: player 1 plays Nimply(row=5, num_objects=7)
INFO:root:status: <0 3 5 4 9 4>
INFO:root:ply: player 0 plays Nimply(row=1, num_objects=3)
INFO:root:status: <0 0 5 4 9 4>
INFO:root:ply: player 1 plays Nimply(row=5, num_objects=3)
INFO:root:status: <0 0 5 4 9 1>
INFO:root:ply: player 0 plays Nimply(row=2, num_objects=5)
INFO:root:status: <0 0 0 4 9 1>
INFO:root:ply: player 1 plays Nimply(row=3, num_objects=3)
INFO:root:status: <0 0 0

dict_items([(Nimply(row=5, num_objects=1), 6), (Nimply(row=5, num_objects=2), 5), (Nimply(row=5, num_objects=3), 4), (Nimply(row=5, num_objects=4), 3), (Nimply(row=5, num_objects=5), 2), (Nimply(row=5, num_objects=6), 1), (Nimply(row=5, num_objects=7), 0)])
(1, 3, 5, 7, 9, 11)
dict_items([(Nimply(row=0, num_objects=1), 3), (Nimply(row=1, num_objects=1), 3), (Nimply(row=1, num_objects=2), 0), (Nimply(row=1, num_objects=3), 1), (Nimply(row=2, num_objects=1), 3), (Nimply(row=2, num_objects=2), 4), (Nimply(row=2, num_objects=3), 5), (Nimply(row=2, num_objects=4), 6), (Nimply(row=2, num_objects=5), 7), (Nimply(row=3, num_objects=1), 3), (Nimply(row=3, num_objects=2), 0), (Nimply(row=3, num_objects=3), 1), (Nimply(row=3, num_objects=4), 6), (Nimply(row=3, num_objects=5), 7), (Nimply(row=3, num_objects=6), 4), (Nimply(row=3, num_objects=7), 5), (Nimply(row=4, num_objects=1), 3), (Nimply(row=4, num_objects=2), 12), (Nimply(row=4, num_objects=3), 13), (Nimply(row=4, num_objects=4), 14), (Nimply